<a href="https://colab.research.google.com/github/PANDASANG1231/Deeplearning_byHand/blob/main/056_GRU_generate_novel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [30]:
import sys
import time
from google.colab import drive
drive.mount('/content/drive', force_remount=True)
sys.path.append('/content/drive/MyDrive/Colab Notebooks/')

Mounted at /content/drive


In [31]:
!git clone --branch main https://github.com/PANDASANG1231/Deeplearning_byHand.git

fatal: destination path 'Deeplearning_byHand' already exists and is not an empty directory.


In [32]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


## Functions


### Preprocess Data

In [33]:
import copy
import json
import math
import numpy as np
import pandas as pd
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from collections import Counter

class Vocab:

    def __init__(self, tokens=None, reserved_tokens=None, min_freq=0):

        if tokens is None:
            tokens = []
        if reserved_tokens is None:
            reserved_tokens = []

        counter = count_corpus(tokens)
        self._token_freqs = sorted(counter.items(), key=lambda x: x[1], reverse=True)

        self.idx_to_token = ['<unk>'] + reserved_tokens
        self.token_to_idx = {token: idx for idx, token in enumerate(self.idx_to_token)}
        for token, freq in self._token_freqs:
            if freq < min_freq:
                break
            if token not in self.token_to_idx:
                self.idx_to_token.append(token)
                self.token_to_idx[token] = len(self.idx_to_token) - 1

    def __len__(self):
        return len(self.idx_to_token)

    def __getitem__(self, tokens):
        if not isinstance(tokens, (list, tuple)):
            return self.token_to_idx.get(tokens, self.unk)
        return [self.__getitem__(token) for token in tokens]

    def to_tokens(self, indices):
        if not isinstance(indices, (list, tuple)):
            return self.idx_to_token[indices]
        return [self.idx_to_token[index] for index in indices]

    @property
    def unk(self):
        return 0

    @property
    def token_freqs(self):
        return self._token_freqs

def load_txt(txt_file_path):

  rst = []
  with open(txt_file_path) as f:
      for i in f.readlines():
          rst.append(i)

  return rst

def tokenize(lines, type_="char", lower_case=True):

    if lower_case:
        lines = [line.lower() for line in lines]
    if type_ == "word":
        return [line.split() for line in lines]
    elif type_ == "char":
        return [list(line) for line in lines]
    else:
        print("unknown type, type should be in ['char', 'word']")

def count_corpus(lines):

    token_list = [token for line in lines for token in line]
    c = Counter(token_list)

    return c

def load_time_machine_corpus(txt_file_path, min_freq=0, max_corpus=-1, lower_case=True, type_="char"):
    rst = load_txt(txt_file_path)
    tokens = tokenize(rst, type_=type_, lower_case=lower_case)

    vocab = Vocab(tokens, min_freq=min_freq)
    corpus = [vocab[token] for line in tokens for token in line]

    if max_corpus > 0:
        corpus = corpus[:max_corpus]

    return corpus, vocab

### Define Dataloader

In [34]:
from torch._C import device
from sklearn.utils import shuffle
import torch
from torch import utils
from torch import nn
from torch.nn.functional import one_hot

def handle_corpus(corpus, time_steps, train_size=0.9, random_state=2022):

    data = [corpus[i: i+time_steps] for i in range(len(corpus) - time_steps)]
    data_frame = pd.DataFrame([]).assign(
        X = data[:-1],
        y = data[1:]
    )
    train_data_frame, valid_data_frame = train_test_split(
        data_frame, train_size=train_size, shuffle=True, random_state=random_state
        )

    return train_data_frame, valid_data_frame

class seqDataSet(utils.data.Dataset):

    def __init__(self, data_frame, vocab, type_="char"):

        self.data_frame = data_frame
        self.vocab = vocab
        self.X = data_frame["X"].values
        self.y = data_frame["y"].values
        self.type_ = type_
        self.num_classes = len(vocab.token_to_idx.keys())

    def __len__(self):

        return len(self.y)

    def __getitem__(self, idx):

        item_X, item_y = torch.LongTensor(self.X[idx]), torch.LongTensor(self.y[idx])

        if self.type_ == "char":
            return one_hot(item_X, self.num_classes), one_hot(item_y, self.num_classes)
        else:
            return item_X, one_hot(item_y, self.num_classes)


### Define Model

In [35]:
class languageModel(nn.Module):
    
    def __init__(self, output_size, input_size, hidden_size=128, num_layers=2):

        super(languageModel, self).__init__()
        
        self.backbone = nn.GRU(
                input_size=input_size, 
                hidden_size=hidden_size,
                num_layers=num_layers,
                dropout=0.3,
                batch_first=True,
            )
        
        self.tail = nn.Linear(hidden_size, output_size)
        
    def forward(self, x, h=None):
        if h is None:
            x, h = self.backbone(x.float())
        else:
            x, h = self.backbone(x.float(), h)
        
        x = self.tail(x)
        
        return x.reshape(-1, x.shape[-1]), h


### Train Function

In [36]:
from transformers import get_linear_schedule_with_warmup


def train_epoch(model, loss, optimizer, scheduler, train_data_iter, valid_data_iter, device):

    eval_accu = [0, 0]
    time_accu = [0, 0]
    state = None

    for batch_X, batch_y in tqdm(train_data_iter, total=len(train_data_iter)):

        model.to(torch.float32)
        model.train()
        
        time_ = time.time()
        batch_X, batch_y = batch_X.to(device=device), batch_y.to(device=device)
        batch_y = batch_y.reshape(-1, batch_y.shape[-1]).to(torch.float32)

        ## 注意这里是将最近的一个state移出计算图，后面model(batch_X, state)后
        ## state会重新进入计算图，但是state之间不会循环反向传递
        if state is None:
            pass
        elif not isinstance(state, list) and not isinstance(state, tuple):
            state.detach_()
        else:
            for s in state:
                s.detach_()

        batch_y_hat, state = model(batch_X, state)
        batch_loss = loss(batch_y_hat, batch_y)
      
        optimizer.zero_grad()
        batch_loss.backward()
        optimizer.step()
        scheduler.step()
        time_ = time.time() - time_

        with torch.no_grad():
            n = batch_X.shape[0] * batch_X.shape[1]
            eval_accu = [sum(x) for x in zip(*[eval_accu, [n, batch_loss * n]])]

    train_metrics = {
        "train_perplexity": math.exp(eval_accu[1] / eval_accu[0]),
        "train_loss": eval_accu[1] / eval_accu[0],
        }

    eval_accu = [0, 0]
    time_accu = [0, 0]
    state = None

    for batch_X, batch_y in tqdm(valid_data_iter, total=len(valid_data_iter)):

        model.to(torch.float32)
        model.eval()
        batch_X, batch_y = batch_X.to(device=device), batch_y.to(device=device)
        batch_y = batch_y.reshape(-1, batch_y.shape[-1]).to(torch.float32)

        batch_y_hat, state = model(batch_X, state)
        batch_loss = loss(batch_y_hat, batch_y)

        n = batch_X.shape[0] * batch_X.shape[1]
        eval_accu = [sum(x) for x in zip(*[eval_accu, [n, batch_loss * n]])]

    valid_metrics = {
        "valid_perplexity": math.exp(eval_accu[1] / eval_accu[0]),
        "valid_loss": eval_accu[1] / eval_accu[0],
        }


    return train_metrics, valid_metrics


def train(model, 
          train_data_iter, 
          valid_data_iter, 
          epochs=50,
          warmup_prop=0.1,
          lr=1e-2,
          device=torch.device("cuda"),
          type_="char"
          ):

    model.to(device)
    optimizer = torch.optim.SGD(model.parameters(), lr=lr)
    loss = nn.CrossEntropyLoss(reduction='mean')
    
    num_training_steps = int(epochs * len(train_data_iter))
    num_warmup_steps = int(warmup_prop * num_training_steps)
    scheduler = get_linear_schedule_with_warmup(
        optimizer, num_warmup_steps, num_training_steps
    )

    for epoch in range(epochs):

        train_metrics, valid_metrics = train_epoch(model, 
                                                   loss, 
                                                   optimizer, 
                                                   scheduler, 
                                                   train_data_iter, 
                                                   valid_data_iter, 
                                                   device)
        lr = scheduler.get_last_lr()[0]
        print(f"Epoch {epoch + 1:02d}/{epochs:02d} \t lr={lr:.1e}\t")
        print(train_metrics, valid_metrics)

        if type_ == "char":
            print(inference(model, vocab, "time traveller ", seq_num=100))
        elif type_ == "word":
            print(inference_word(model, vocab, ["time", "traveller",], seq_num=100))


def inference(model_, vocab, start, seq_num=10):
    
    ## State一定要留下来，作为每一步预测的输入。否则结果会一样
    ## Just like "Alex like eat like eat like eat...."

    model = copy.deepcopy(model_).to(torch.device("cpu"))
    state, rst = None, []
    for char in start:

        input = one_hot(torch.LongTensor([vocab[char]]), len(vocab.idx_to_token))
        pred, state = model(input, state)
        rst.append(char)

    cur_char = char
    for i in range(seq_num):

        input = one_hot(torch.LongTensor([vocab[cur_char]]), len(vocab.idx_to_token))
        input.to(torch.device("cpu"))
        pred, state = model(input, state)
        pred = torch.nn.Sigmoid()(pred).argmax()
        cur_char = vocab.idx_to_token[pred.item()]
        rst.append(cur_char)
    
    return "".join(rst)


def inference_word(model_, vocab, start, seq_num=10):

    model = copy.deepcopy(model_).to(torch.device("cpu"))
    state, rst = None, []
    for char in start:

        input = torch.LongTensor([vocab[char]])
        pred, state = model(input, state)
        rst.append(char)

    cur_char = char
    for i in range(seq_num):
        try:
            input = torch.LongTensor([vocab[cur_char]])
            input.to(torch.device("cpu"))
            pred, state = model(input, state)
            pred = torch.nn.Sigmoid()(pred).argmax()
            cur_char = vocab.idx_to_token[pred.item()]
            rst.append(cur_char)
        except:
            print(pred.item())
            print(pred)
            return
    
    return " ".join(rst)

## Train the model using 'char' split

In [37]:

txt_file_path = "/content/drive/MyDrive/Colab Notebooks/data/time_machine.txt" 
corpus, vocab = load_time_machine_corpus(txt_file_path, min_freq=90, max_corpus=10000, type_="char")

train_data, valid_data = handle_corpus(corpus, time_steps=35)
train_dataset = seqDataSet(train_data, vocab, type_="char")
valid_dataset = seqDataSet(valid_data, vocab, type_="char")
train_dataloader = utils.data.DataLoader(dataset=train_dataset, batch_size=32, drop_last=True,
                                         num_workers=4, pin_memory=False, shuffle=True)
valid_dataloader = utils.data.DataLoader(dataset=valid_dataset, batch_size=32, drop_last=True,
                                         num_workers=4, pin_memory=False, shuffle=False)


language_model = languageModel(output_size=38, input_size=38)
train(language_model, train_dataloader, valid_dataloader, lr=1, epochs=100, type_="char")


100%|██████████| 31/31 [00:00<00:00, 42.76it/s]


Epoch 01/100 	 lr=1.0e-01	
{'train_perplexity': 28.106782938493335, 'train_loss': tensor(3.3360, device='cuda:0')} {'valid_perplexity': 21.160801575230966, 'valid_loss': tensor(3.0522, device='cuda:0', grad_fn=<DivBackward0>)}
time traveller                                                                                                     


100%|██████████| 31/31 [00:01<00:00, 28.03it/s]


Epoch 02/100 	 lr=2.0e-01	
{'train_perplexity': 20.898883568024093, 'train_loss': tensor(3.0397, device='cuda:0')} {'valid_perplexity': 20.849255281125785, 'valid_loss': tensor(3.0373, device='cuda:0', grad_fn=<DivBackward0>)}
time traveller                                                                                                     


100%|██████████| 31/31 [00:00<00:00, 37.04it/s]


...

Epoch 98/100 	 lr=2.2e-02	
{'train_perplexity': 1.616883047363103, 'train_loss': tensor(0.4805, device='cuda:0')} {'valid_perplexity': 1.5467014448585321, 'valid_loss': tensor(0.4361, device='cuda:0', grad_fn=<DivBackward0>)}
time traveller returns
 iv time travelling
 v in the golden age
 vi the sunset of mankind
 vii a sudden shock
 viii


100%|██████████| 31/31 [00:00<00:00, 45.75it/s]


Epoch 99/100 	 lr=1.1e-02	
{'train_perplexity': 1.6139226538065239, 'train_loss': tensor(0.4787, device='cuda:0')} {'valid_perplexity': 1.5464151273894726, 'valid_loss': tensor(0.4359, device='cuda:0', grad_fn=<DivBackward0>)}
time traveller returns
 iv time travelling
 v in the golden age
 vi the sunset of mankind
 vii a sudden shock
 viii


100%|██████████| 31/31 [00:00<00:00, 44.89it/s]


Epoch 100/100 	 lr=0.0e+00	
{'train_perplexity': 1.6128556016145201, 'train_loss': tensor(0.4780, device='cuda:0')} {'valid_perplexity': 1.546292725793556, 'valid_loss': tensor(0.4359, device='cuda:0', grad_fn=<DivBackward0>)}
time traveller returns
 iv time travelling
 v in the golden age
 vi the sunset of mankind
 vii a sudden shock
 viii


In [38]:
print(inference(language_model, vocab, "time traveller will", seq_num=1000))

time traveller will they could move a little up and down,” said the medical man.
“our ancestors had no great tolerance for anachronisms.”

“one might get one’s greek from the very lips of homer and plato,” the
very young man.

“that shall travel indifferently in any direction of space and time, as
the driver determines.”

filby contented himself with laughter.

“but i have experimental verification,” said the time traveller.

“it’s against reason,” said filby.

“can a cube that does not last for any time at all, have a real
existence. they taught you that? neither has a mathematical plane.
these things are mere abstractions.”

“that is all right,” said the psychologist.

“nor, having only length, breadth, and thickness, and—duration. but through a natural
infirmity of the flesh, which i will explain to you in a moment, we
incline to overlook this fact. there are really four dimensions for some time. some of my results are curious. for
instance, here is a portrait of a man at eight year

## Train the model using 'word' split

In [39]:
class languageModelWord(nn.Module):
    
    def __init__(self, output_size, embed_size, hidden_size=128, num_layers=2):

        super(languageModelWord, self).__init__()

        self.head = nn.Embedding(len(vocab.idx_to_token), embed_size)
        
        self.backbone = nn.GRU(
                input_size=embed_size, 
                hidden_size=hidden_size,
                num_layers=num_layers,
                dropout=0.3,
                batch_first=True,
            )
        
        self.tail = nn.Linear(hidden_size, output_size)
        
    def forward(self, x, h=None):

        x = self.head(x)

        if h is None:
            x, h = self.backbone(x.float())
        else:
            x, h = self.backbone(x.float(), h)
        
        x = self.tail(x)
        
        return x.reshape(-1, x.shape[-1]), h

In [40]:
txt_file_path = "/content/drive/MyDrive/Colab Notebooks/data/time_machine.txt" 
corpus, vocab = load_time_machine_corpus(txt_file_path, min_freq=1, max_corpus=10000, type_="word")

train_data, valid_data = handle_corpus(corpus, time_steps=35)
train_dataset = seqDataSet(train_data, vocab, type_="word")
valid_dataset = seqDataSet(valid_data, vocab, type_="word")
train_dataloader = utils.data.DataLoader(dataset=train_dataset, batch_size=32, drop_last=True,
                                         num_workers=4, pin_memory=False, shuffle=True)
valid_dataloader = utils.data.DataLoader(dataset=valid_dataset, batch_size=32, drop_last=True,
                                         num_workers=4, pin_memory=False, shuffle=False)

output_size = valid_dataset.num_classes
print(output_size)

language_model = languageModelWord(output_size=output_size, embed_size=1000)

train(language_model, train_dataloader, valid_dataloader, lr=1, epochs=40, type_="word")


7294


100%|██████████| 31/31 [00:01<00:00, 15.97it/s]


Epoch 01/40 	 lr=2.5e-01	
{'train_perplexity': 2786.719914661251, 'train_loss': tensor(7.9326, device='cuda:0')} {'valid_perplexity': 902.0340006176831, 'valid_loss': tensor(6.8047, device='cuda:0', grad_fn=<DivBackward0>)}
time traveller of the of the of the of the of the of the of the of the of the of the of the of the of the of the of the of the of the of the of the of the of the of the of the of the of the of the of the of the of the of the of the of the of the of the of the of the of the of the of the of the of the of the of the of the of the of the of the of the of the of the


100%|██████████| 31/31 [00:01<00:00, 15.95it/s]


Epoch 02/40 	 lr=5.0e-01	
{'train_perplexity': 641.5271153783534, 'train_loss': tensor(6.4639, device='cuda:0')} {'valid_perplexity': 441.5997962890852, 'valid_loss': tensor(6.0904, device='cuda:0', grad_fn=<DivBackward0>)}
time traveller was a little of a little little i was had a little of a little of a little of a little of a little of a little of a little of a little of a little of a little of a little of a little of a little of a little of a little of a little of a little of a little of a little of a little of a little of a little of a little of a little of a little of a little of a little of a little of a little of a little of


...

Epoch 38/40 	 lr=5.6e-02	
{'train_perplexity': 1.3443904684080634, 'train_loss': tensor(0.2959, device='cuda:0')} {'valid_perplexity': 1.2559225158213485, 'valid_loss': tensor(0.2279, device='cuda:0', grad_fn=<DivBackward0>)}
time traveller came back, and filby’s anecdote collapsed. ii. the machine the thing the time traveller held in his hand was a glittering metallic framework, scarcely larger than a small clock, and very delicately made. there was ivory in it, and some transparent crystalline substance. and now i must be explicit, for this that follows—unless his explanation is to be accepted—is an absolutely unaccountable thing. he took one of the small octagonal tables that were scattered about the room, and set it in front of the fire, with two legs on the hearthrug. on this table he placed the mechanism. then he


100%|██████████| 31/31 [00:01<00:00, 16.09it/s]


Epoch 39/40 	 lr=2.8e-02	
{'train_perplexity': 1.3421057675939132, 'train_loss': tensor(0.2942, device='cuda:0')} {'valid_perplexity': 1.2550999746227216, 'valid_loss': tensor(0.2272, device='cuda:0', grad_fn=<DivBackward0>)}
time traveller came back, and filby’s anecdote collapsed. ii. the machine the thing the time traveller held in his hand was a glittering metallic framework, scarcely larger than a small clock, and very delicately made. there was ivory in it, and some transparent crystalline substance. and now i must be explicit, for this that follows—unless his explanation is to be accepted—is an absolutely unaccountable thing. he took one of the small octagonal tables that were scattered about the room, and set it in front of the fire, with two legs on the hearthrug. on this table he placed the mechanism. then he


100%|██████████| 31/31 [00:01<00:00, 16.16it/s]

Epoch 40/40 	 lr=0.0e+00	
{'train_perplexity': 1.3403443783511677, 'train_loss': tensor(0.2929, device='cuda:0')} {'valid_perplexity': 1.25490436227179, 'valid_loss': tensor(0.2271, device='cuda:0', grad_fn=<DivBackward0>)}
time traveller came back, and filby’s anecdote collapsed. ii. the machine the thing the time traveller held in his hand was a glittering metallic framework, scarcely larger than a small clock, and very delicately made. there was ivory in it, and some transparent crystalline substance. and now i must be explicit, for this that follows—unless his explanation is to be accepted—is an absolutely unaccountable thing. he took one of the small octagonal tables that were scattered about the room, and set it in front of the fire, with two legs on the hearthrug. on this table he placed the mechanism. then he


In [48]:
inference_word(language_model, vocab, ["Alex", "will", "have", "dinner"], seq_num=1000)

'Alex will have dinner at seven if he’s not back. says he’ll explain when he comes.” “it seems a pity to let the dinner spoil,” said the editor of a well-known daily paper; and thereupon the doctor rang the bell. the psychologist was the only person besides the doctor and myself who had attended the previous dinner. the other men were blank, the editor aforementioned, a certain journalist, and another—a quiet, shy man with a beard—whom i didn’t know, and who, as far as my observation went, never opened his mouth all the evening. there was some speculation at the dinner-table about the time traveller’s absence, and i suggested time travelling, in a half-jocular spirit. the editor wanted that explained to him, and the psychologist volunteered a wooden account of the “ingenious paradox and trick” we had witnessed that day week. he was in the midst of his exposition when the door from the corridor opened slowly and without noise. i was facing the door, and saw it first. “hallo!” i said. “a